# Environment Set-up, Pre-processing and Hand-crafted Features Extraction

In [ ]:
# Environment Set-up
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
sess = tf.Session(config=tf_config)

set_session(sess)
import gensim
import keras
import re
from string import punctuation
import numpy as np
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from tqdm import tqdm
from sklearn import feature_extraction
from pandas import DataFrame
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
np.random.seed(1003)
import numpy as np
from collections import defaultdict
import nltk
import numpy as np


from keras.models import Model
from keras.layers import Layer, Input, Dense, Concatenate, Conv2D, Reshape, MaxPooling1D, Flatten, BatchNormalization, Activation, Dropout, Embedding
import nltk
_wnl = nltk.WordNetLemmatizer()


# Specify the folder locations
W2V_DIR = '/home/students/y2886wan/W/GoogleNews-vectors-negative300.bin'
DATA_DIR = 'data/'
#EMB_DIR = '/home/students/y2886wan/glove_data/glove.6B.50d.txt'

# Hyperparameters 
MAX_SENT_LEN_1 = 16
MAX_SENT_LEN_2 = 512
LSTM_DIM = 16
MAX_VOCAB_SIZE=26562
EMBEDDING_DIM = 300
BATCH_SIZE = 128
N_EPOCHS = 200


# Loading data
trainData_Heading = pd.read_csv("/home/students/y2886wan/Data/train_data.csv")
valData_Heading = pd.read_csv("/home/students/y2886wan/Data/validation_data.csv")
trainData_Bodies = pd.read_csv("/home/students/y2886wan/Data/article_body_texts.csv")

# Stop word
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']

# Data generater
def generate_data(Heading,Bodies):

    Data = pd.merge(Heading, Bodies, how='left', on='Body ID')

    Data = Data.sample(frac=1, random_state=10) # Shuffle the rows
    Data.reset_index(inplace=True, drop=True)

    Data['Headline'] = Data['Headline'].apply(lambda x: str(x))
    Data['articleBody'] = Data['articleBody'].apply(lambda x: str(x))
    
    return Data

def normalize_word(w):
    return _wnl.lemmatize(w).lower()

# Cleaning data
def text_to_wordlist(text, remove_stop_words=True, stem_words=False):
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" U S ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)


# Processing headline and bodies
def process_questions(question_list, questions, question_list_name, dataframe):
    for question in questions:
        question_list.append(text_to_wordlist(question))
        if len(question_list) % 100000 == 0:
            progress = len(question_list)/len(dataframe) * 100
            print("{} is {}% complete.".format(question_list_name, round(progress, 1)))


            
# Overlap features extraction
def get_pairwise_word_to_doc_freq(headlines, bodies):
    """
    Get pairwise word to document frequency.
    For index i, if sentence i in headlines and sentence i in bodies both
    container word w, then w is counted only once.
    Returns a dictionary mapping words to number of sentence pairs the word appears in.
    """
    word_to_doc_cnt = defaultdict(int)

    for s1, s2 in zip(headlines, bodies):
        unique_tokens = set(s1) | set(s2)
        for t in unique_tokens:
            word_to_doc_cnt[t] += 1

    return word_to_doc_cnt

def get_pairwise_overlap_features(headlines, bodies, word_to_doc_cnt):
    """
    Get overlap, idf weighted overlap, overlap excluding stopwords, and idf weighted overlap excluding stopwords.
    """
    stoplist = set(stop_words)
    num_docs = len(headlines)
    overlap_feats = []

    for s1, s2 in zip(headlines, bodies):
        tokens_a_set, tokens_b_set = set(s1), set(s2)
        intersect = tokens_a_set & tokens_b_set
        overlap = len(intersect) / len(tokens_a_set) 
        idf_intersect = sum(np.math.log(num_docs / word_to_doc_cnt[w]) for w in intersect)
        idf_weighted_overlap = idf_intersect / (len(tokens_a_set) + len(tokens_b_set))

        tokens_a_set_no_stop = set(w for w in s1 if w not in stoplist)
        tokens_b_set_no_stop = set(w for w in s2 if w not in stoplist)
        intersect_no_stop = tokens_a_set_no_stop & tokens_b_set_no_stop
        overlap_no_stop = len(intersect_no_stop) / (len(tokens_a_set_no_stop) + len(tokens_b_set_no_stop))
        idf_intersect_no_stop = sum(np.math.log(num_docs / word_to_doc_cnt[w]) for w in intersect_no_stop)
        idf_weighted_overlap_no_stop = idf_intersect_no_stop / (len(tokens_a_set_no_stop) + len(tokens_b_set_no_stop))
        overlap_feats.append([overlap, idf_weighted_overlap, overlap_no_stop, idf_weighted_overlap_no_stop])

    return overlap_feats




# Refuting features extraction
def refuting_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        # 'refute',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_headline = get_tokenized_lemmas(clean_headline)
        features = [1 if word in clean_headline else 0 for word in _refuting_words]
        features =list(features)
        X.append(features)
    return X



# Polarity features extraction
def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric

    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()

def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]

def polarity_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]

    def calculate_polarity(text):
        tokens = get_tokenized_lemmas(text)
        return sum([t in _refuting_words for t in tokens]) % 2
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        features = []
        features.append(calculate_polarity(clean_headline))
        features.append(calculate_polarity(clean_body))
        X.append(features)
    return np.array(X)


# Concatenate training data and validation data
test_count = len(valData_Heading)
trainData = generate_data(trainData_Heading,trainData_Bodies)
valData = generate_data(valData_Heading,trainData_Bodies)
frames=[trainData,valData]
trainData = pd.concat(frames)


# Processing headline and bodies
train_headling = []
process_questions(train_headling, trainData.Headline, 'train_headling', trainData)
train_bodies = []
process_questions(train_bodies, trainData.articleBody, 'train_bodies', trainData)

# Generating hand-crafted features
cnt = get_pairwise_word_to_doc_freq(train_headling, train_bodies)
overlap_features = get_pairwise_overlap_features(train_headling, train_bodies, cnt)
refuting_feature = refuting_features(train_headling, train_bodies)
polarity_feature = polarity_features(train_headling, train_bodies)

word_seq = train_headling + train_bodies
word_seq = [' '.join(word_tokenize(q)[:MAX_SENT_LEN_2]) for q in word_seq]


# Tokenizer, transform words into interger identifier, create a dictionary
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='!"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(word_seq)
print("Number of words in vocabulary:", len(tokenizer.word_index))


# Transform all document into vectors using dictionary
X = tokenizer.texts_to_sequences(word_seq)
X = pad_sequences(X, maxlen=MAX_SENT_LEN_2, padding='post', truncating='post')

# Separate headline and bodies
X_q1 = X[:len(X)//2]
X_q2 = X[len(X)//2:]
del X

# Reducing dimension of headline from 512 to 16
X_q1 = np.delete(X_q1, range(MAX_SENT_LEN_1, MAX_SENT_LEN_2), axis = 1)

# Transform each stance into vector
Stance_list = [' '.join(word_tokenize(x)[:MAX_SENT_LEN_2]) for x in trainData['Stance']]
wordstance=[]
for i in Stance_list:
    if i=='unrelated':
        wordstance.append([0,0,0,1])
    if i=='discuss':
        wordstance.append([0,0,1,0])
    if i=='agree':
        wordstance.append([0,1,0,0])
    if i=='disagree':
        wordstance.append([1,0,0,0])


# unpacked = [x[0] for x in y_stance]
y = np.array(wordstance)

# # Split Train and Test Set 

X_train_q1, X_test_q1, X_train_q2, X_test_q2, overlap_train, overlap_test, refuting_train, refuting_test, polarity_train, polarity_test, y_train, y_test = train_test_split(X_q1, X_q2, overlap_features, refuting_feature, polarity_feature, y, random_state=10, test_size=test_count)

# Transform array to list
polarity_train = [i.tolist() for i in polarity_train]
polarity_test = [i.tolist() for i in polarity_test]

# Concatenate features for train
features_train = [[x+y+z] for x,y,z in zip(overlap_train, refuting_train, polarity_train)]
feature_train = [x[0] for x in features_train]

# Concatenate features for test
features_test = [[x+y+z] for x,y,z in zip(overlap_test, refuting_test, polarity_test)]
feature_test = [x[0] for x in features_test]


# Embedding Matrix

In [ ]:
# Word2Vec model
embeddings = gensim.models.KeyedVectors.load_word2vec_format(W2V_DIR, binary=True)

In [ ]:
# GloVe embedding
'''
# Load GloVe word embeddings
# Download Link: https://nlp.stanford.edu/projects/glove/
print("[INFO]: Reading Word Embeddings ...")
# Data path
embeddings = {}
f = open(EMB_DIR)
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    embeddings[word] = vector
f.close()
'''

In [ ]:
# Words embedding
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index) + 1, EMBEDDING_DIM))  # +1 is because the matrix indices start with 0
for word, i in tokenizer.word_index.items():  # i=0 is the embedding for the zero padding
    try:
        embeddings_vector = embeddings[word]
    except KeyError:
        embeddings_vector = None
    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector

del embeddings

# LSTM Model

In [ ]:
# LSTM
input_1 = Input(shape=(MAX_SENT_LEN_1, ), name='q1_input') #shape: a vector
# Common embedding lookup layer
emb_look_up = Embedding(input_dim=MAX_VOCAB_SIZE,
                        output_dim=EMBEDDING_DIM,
                        weights = [embeddings_matrix], 
                        trainable=False, 
                        mask_zero=False,
                        name='q_embedding_lookup')

emb_1 = emb_look_up(input_1)

LSTM_Layer_1 = LSTM(LSTM_DIM, return_sequences=False, name='lstm_layer_1')
LSTM_Out = LSTM_Layer_1(emb_1)
Dense_layer_1 = Dense(8, activation='sigmoid', name='output_layer_1')
Dense_1_out = Dense_layer_1(LSTM_Out)
Dense_layer_2 = Dense(4, activation='sigmoid', name='output_layer_2')
Dense_2_out = Dense_layer_2(Dense_1_out)

In [ ]:
# LSTM
input_2 = Input(shape=(MAX_SENT_LEN_2, ), name='q2_input')

emb_2 = emb_look_up(input_2)

LSTM_Layer_2 = LSTM(LSTM_DIM, return_sequences=False, name='lstm_layer_2')
LSTM_Out_2 = LSTM_Layer_2(emb_2)
Dense_1 = Dense(256, activation='sigmoid', name='output_layer_3')
Dense_Out_1 = Dense_1(LSTM_Out_2)
Dense_2 = Dense(128, activation='sigmoid', name='output_layer_4')
Dense_Out_2 = Dense_2(Dense_Out_1)
Dense_3 = Dense(64, activation='sigmoid', name='output_layer_5')
Dense_Out_3 = Dense_3(Dense_Out_2)
Dense_4 = Dense(32, activation='sigmoid', name='output_layer_6')
Dense_Out_4 = Dense_4(Dense_Out_3)
Dense_5 = Dense(16, activation='sigmoid', name='output_layer_7')
Dense_Out_5 = Dense_5(Dense_Out_4)
Dense_6 = Dense(8, activation='sigmoid', name='output_layer_8')
Dense_Out_6 = Dense_6(Dense_Out_5)
Dense_7 = Dense(4, activation='sigmoid', name='output_layer_9')
Dense_Out_7 = Dense_7(Dense_Out_6)

In [ ]:
# With batch-normalization, the output of a previous layer is mu-sigma normalized, 
# before it is fed into the next layer. 
# For feed-forward networks, batch-normalization is carried out 

# Additional features input
input_3 = Input(shape=(21, ), name='feature_input')

# Merge headline, bodies and hand-crafted features
merged = Concatenate(name='headling_bodies_features_concat')([Dense_2_out, Dense_Out_7, input_3])

# Dense layers
dense_1 = Dense(units=10, name='headling_bodies_features_dense')(merged)
bn_1 = BatchNormalization(name='batchnorm')(dense_1)
relu_1 = Activation(activation='relu', name='relu_activation')(bn_1)
dense_1_dropout = Dropout(0.2, name='dense_dropout')(relu_1)

output_prob = Dense(units=4, activation='softmax', name='output_layer')(dense_1_dropout)

model = Model(inputs=[input_1, input_2, input_3], outputs=output_prob, name='text_pair_cnn')
model.summary()


In [ ]:
import keras.backend as K


from keras import metrics
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', metrics.categorical_accuracy])

In [ ]:
# Training LSTM and CNN model
model.fit(x = [X_train_q1, X_train_q2, np.array(feature_train)], 
          y = y_train, 
          batch_size=BATCH_SIZE, 
          epochs=N_EPOCHS, 
          validation_data=([X_test_q1, X_test_q2, np.array(feature_test)], y_test))

In [ ]:
model.save('my_model_try_128_200epoch.h5')

In [ ]:
prediction = model.predict([X_test_q1, X_test_q2, np.array(feature_test)])

# Evaluation

In [ ]:
correct = {0: 0, 1:0, 2: 0, 3: 0}
pred_count = {0: 0, 1:0, 2: 0, 3: 0}
true_count = {0: 0, 1:0, 2: 0, 3: 0}
prediction2 = np.argmax(prediction, axis=-1)
y_train2 = np.argmax(y_test, axis=-1)
for p, t in zip(prediction2, y_train2):
    if p == t:
        correct[p] += 1
    pred_count[p] += 1
    true_count[t] += 1

In [ ]:
pred_count

In [ ]:
true_count

In [ ]:
for k in correct:
    c, t, p = correct[k], true_count[k], pred_count[k]
    pre = c / p
    rec = c / t
    f1 = 2 * pre * rec / (pre + rec)
    print("precision at {}: {}".format(k, pre))
    print("recall at {}: {}".format(k, rec))
    print("f1 at {}: {}".format(k, f1))

# Prediction

In [ ]:
# Input test data and transform it into dictionary
testData_Heading = pd.read_csv("/home/students/y2886wan/Data/test_data.csv")
testData = pd.merge(testData_Heading, trainData_Bodies, how='left', on='Body ID')
testData['Headline'] = testData['Headline'].apply(lambda x: str(x))
testData['articleBody'] = testData['articleBody'].apply(lambda x: str(x))

# Generating the features for test data 
test_headling = []
process_questions(test_headling, testData.Headline, 'test_headling', testData)
test_bodies = []
process_questions(test_bodies, testData.articleBody, 'testData_bodies', testData)

### overlap
cnt_t = get_pairwise_word_to_doc_freq(test_headling, test_bodies)
overlap_features_t = get_pairwise_overlap_features(test_headling, test_bodies, cnt_t)

### refuting
refuting_feature_t = refuting_features(test_headling, test_bodies)

### polarity
polarity_feature_t = polarity_features(test_headling, test_bodies)
polarity_t = [i.tolist() for i in polarity_feature_t]


features_t = [[x+y+z] for x,y,z in zip(overlap_features_t, refuting_feature_t, polarity_t)]
feature_t = [x[0] for x in features_t]



# Tokenizer
word_seq_t = test_headling + test_bodies
word_seq_t = [' '.join(word_tokenize(q)[:MAX_SENT_LEN_2]) for q in word_seq_t]
tokenizer_t = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='!"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n')
tokenizer_t.fit_on_texts(word_seq_t)

# Transform test data into matrix using dictionary
XT = tokenizer_t.texts_to_sequences(word_seq_t)
XT = pad_sequences(XT, maxlen=MAX_SENT_LEN_2, padding='post', truncating='post')

X_q1_t = XT[:len(XT)//2]
X_q2_t = XT[len(XT)//2:]
X_q1_t = np.delete(X_q1_t, range(MAX_SENT_LEN_1, MAX_SENT_LEN_2), axis = 1)
del XT

In [ ]:
# Model prediction
prediction2 = model.predict(x = [X_q1_t, X_q2_t, np.array(feature_t)])

In [ ]:
# Choosing the highest probability position of each vector
predicted = [[np.argmax(x)] for x in prediction2]

In [ ]:
# transform all position into string (four classes) 
wordstance=[]
for i in predicted:
    if i==[0]:
        wordstance.append("disagree")
    if i==[1]:
        wordstance.append("agree")
    if i==[2]:
        wordstance.append("discuss")
    if i==[3]:
        wordstance.append("unrelated")


In [ ]:
# Prediction save as CSV file
wordseries=pd.Series(wordstance) # all stance
testData = pd.read_csv("/home/students/y2886wan/Data/test_data.csv")
testData['Stance'] = wordseries.values
results = testData.to_csv('answer.csv',index=False, encoding='utf-8')